# Подготовка данный

## Загрузка

In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device=torch.device("cpu")

<h3> Подгрузим данные

In [4]:
sents=pd.read_json("../Data/Petitions_prepared.json")

In [5]:
sents.head()

,public_petition_text,reason_category
0,"[[снег, дорога]]",Благоустройство
1,"[[очистить, кабельный, киоск, реклама]]",Благоустройство
2,"[[просить, убрать, всё, дерево, кустарник, кот...",Благоустройство
3,"[[неудовлетворительный, состояние, парадный, н...",Содержание МКД
4,[[граффити]],Благоустройство


In [6]:
sents.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59889 entries, 0 to 59888
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   public_petition_text  59889 non-null  object
 1   reason_category       59889 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [7]:
for category in sents["reason_category"].unique():
    print(category,sents["reason_category"].isin([category]).sum())

Благоустройство 35933
Содержание МКД 14895
Проблемы с торговлей, рекламой, информационными стендами 2755
Состояние МКД 2562
Нарушение подачи и отведения горячей и холодной воды 1331
Нарушение порядка пользования общим имуществом 2413


## Превратим наши предложения и категории в числа

<h3> Создадим словари для категорий

In [8]:
cat2idx={"Благоустройство":0,
         "Содержание МКД":1,
         "Проблемы с торговлей, рекламой, информационными стендами":2,
         "Состояние МКД":3,
         "Нарушение подачи и отведения горячей и холодной воды":4,
         "Нарушение порядка пользования общим имуществом":5}
idx2cat={v: k for k, v in cat2idx.items()}

<h3> Подгрузим уже сформированный список слов

In [9]:
idx=pd.read_csv("../Data/emb_indexis.tsv",delimiter="\t")

In [10]:
idx

,Unnamed: 0,0
0,out_of_sentence,out_of_sentence
1,снег,снег
2,дорога,дорога
3,очистить,очистить
4,кабельный,кабельный
...,...,...
2711,допустить,допустить
2712,неужели,неужели
2713,моховой,моховой
2714,войти,войти


<h3> Создадим словарь слов для дальнейшего эмбединга

In [11]:
words=idx["0"].iloc[1:].unique()

In [12]:
word2idx=dict(zip(words,range(1,len(words)+1)))
idx2word={v: k for k, v in word2idx.items()}
vocab_size = len(word2idx)

<h3> Привратим слова в числа и объединим предложения

In [13]:
def convert_words(nested_words):
    return [[word2idx[word] for word in inner_list] for inner_list in nested_words]
sents['public_petition_text'] = sents['public_petition_text'].apply(convert_words)

In [14]:
def unite_sents(nested_words):
    united_sents=nested_words[0]
    for sent in nested_words[1:]: united_sents.extend(sent)
    return united_sents
sents['public_petition_text'] = sents['public_petition_text'].apply( unite_sents)

In [15]:
sents

,public_petition_text,reason_category
0,"[1, 2]",Благоустройство
1,"[3, 4, 5, 6]",Благоустройство
2,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, ...",Благоустройство
3,"[25, 26, 27, 28, 29, 30]",Содержание МКД
4,[31],Благоустройство
...,...,...
59884,"[7, 68, 31]",Благоустройство
59885,"[7, 355, 16, 986]",Благоустройство
59886,"[32, 44, 468, 99, 221]","Проблемы с торговлей, рекламой, информационным..."
59887,"[48, 40, 1598, 699, 242, 594, 67, 8]",Состояние МКД


<h3> Превратим категории в числа

In [16]:
def convert_cat(cat):
    return cat2idx[cat]
sents['reason_category'] = sents['reason_category'].apply(convert_cat)
sents

,public_petition_text,reason_category
0,"[1, 2]",0
1,"[3, 4, 5, 6]",0
2,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, ...",0
3,"[25, 26, 27, 28, 29, 30]",1
4,[31],0
...,...,...
59884,"[7, 68, 31]",0
59885,"[7, 355, 16, 986]",0
59886,"[32, 44, 468, 99, 221]",2
59887,"[48, 40, 1598, 699, 242, 594, 67, 8]",3


## Разобьём данные а также отрегулируем их длинну

<h3> Посмотрим какой длины наши получившиеся предложения

In [17]:
MAX_LEN=0
for edge in [10,30,50,100]:
    ck=0
    for sent in sents['public_petition_text']:
        if len(sent)>=edge:
            ck+=1
        MAX_LEN=max(MAX_LEN,len(sent))
    print(ck)
MAX_LEN

16746
3644
1495
110


232

<h3> Максимальная длина - 232, Сократим длину блоков до 30 слов, чтобы тензоры не были слишком разрежёнными и успевали обратываться

<h3> Разделеним данные на Train, eval, test и создадим даталоудеры

In [18]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torch import nn

<h3> Для того чтобы дисбаланс классов не сказался негативно на модели введём веса и также раазобьём наши данные (создадим функцию)

In [19]:
def make_dataloader(batch_size=16,data=sents,len_of_tensors=30,cat_number=6):

    counts=[0]*6
    for category in data["reason_category"].unique():
            counts[int(category)]=int(data["reason_category"].isin([category]).sum())
    print("Число представителей каждого класса",counts)
    weights=[0]*6
    for i,num in enumerate(counts):
        weights[i]=min(counts)/num
    weights=torch.tensor(weights)
    print("Веса функции потерь",weights)
    
    n = len(data) #число пар
    input_ids = np.zeros((n, len_of_tensors), dtype=np.int32) #создаём обучающие тензоры имеющие в строках списки индексов слов
    target_ids = np.zeros((n), dtype=np.int32) #пока что они заполены нолями


    for idx in range(len(data)): #через цикл перебираем все пары
        input_array=data.iloc[idx]['public_petition_text']
        input_ids[idx,:min(30,len(input_array))]=input_array[:min(30,len(input_array))]
        target_ids[idx]=data.iloc[idx]['reason_category']

    input_train, input_eval, target_train, target_eval = train_test_split(input_ids, target_ids, test_size=0.2, random_state=42)#разбиваем данные на train
    input_val, input_test, target_val, target_test = train_test_split(input_ids, target_ids, test_size=0.5, random_state=42)# test, val
    
    print(torch.LongTensor(input_train))
    train_data = TensorDataset(torch.LongTensor(input_train).to(device), 
                               torch.LongTensor(target_train).to(device)) #преобразуем тензоры в датасет, где эти тензоры будут являтся столбцами датасета

    train_sampler = RandomSampler(train_data) #Если я праавильно понял, то мы рандомно дробим наш датасет и согласно этомк разбиению формируем
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size) #наш Datsloader, специальный класс для итерации по 
                                                                                            # датасету

    val_data = TensorDataset(torch.LongTensor(input_val).to(device), 
                               torch.LongTensor(target_val).to(device)) 
                                                   #по отношению к оценочным и тестовым данным действум тем
    val_dataloader = DataLoader(train_data, batch_size=batch_size)     #же образом
    
    test_data = TensorDataset(torch.LongTensor(input_test).to(device), 
                               torch.LongTensor(target_test).to(device)) 

    test_dataloader = DataLoader(train_data,  batch_size=batch_size)
                                                                                           
    return weights, train_dataloader, val_dataloader, test_dataloader
    



# Инициализация моделей и функций обучения

<h3> Создадим функции для отмера времени при обучении

In [21]:

import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

<h3> Функция оценки качества модели во время обучения (1 эпоха)

In [22]:
def val_epoch(dataloader, model, #на вход передаётся наш dataloader. encoder, decoder
           criterion):                          # а также функция оценки качества (ошибки), которую мы будем минимизировать
    model.eval() #подрубаем режим оценки
    val_loss = 0 
    for data in dataloader: # производим итерацию по батчам наших данных
        
        input_tensor,  target = data

        result = model(input_tensor) #прогоняем батч через инкодер и декодер
        loss = criterion(
           result,target
        )
        val_loss += loss.item() #вычисляем ошибку по эпохе (сумма ошибок)

    return val_loss / len(dataloader) #возвращаем среднюю ошибку

<h3> Функция обучения модели (1 эпоха)

In [23]:
def train_epoch(dataloader, model, optimizer, criterion):                          # а также функция оценки качества (ошибки), которую мы будем минимизировать
    model.train()
    total_loss = 0 
    for data in dataloader: # производим итерацию по батчам наших данных
        input_tensor, target = data
        
        optimizer.zero_grad()
        
        result = model(input_tensor) #прогоняем батч через инкодер и декодер

        loss = criterion(
          result,target
        )
        loss.backward() #проводим обратное распространение, вычисляютя градиенты

        optimizer.step() #корректируем веса нейронок использую аптимизаторы
        total_loss += loss.item() #вычисляем ошибку по эпохе (сумма ошибок)
        

    return total_loss / len(dataloader) #возвращаем среднюю ошибку

<h3> Полная функция обучения, с заданием числа эпох, весов потери, модели и прочих параметров

In [103]:
def train(train_dataloader,val_dataloader, model, n_epochs,loss_weights, learning_rate=0.01, #передаём наши данные, инкодер, декодер, число эпох
               print_every=2,early_stop=False):          #learning_rate(как сильно мы будем изменять веса каждый раз), а также как часто
                                                        #выводить промежуточные результаты
    start = time.time() #стартуем таймер
    print_loss_total = 0  #задаём внутренние переменные
    print_val_loss_total =0
    model_options_best=0
    val_loss_best=math.inf
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss(weight=loss_weights)

    for epoch in range(1, n_epochs + 1): #по количеству эпох итерируем
        loss = train_epoch(train_dataloader, model, optimizer, criterion) #тренируем и вычисляем ошибку
        val_loss=val_epoch(val_dataloader,model,criterion)
        print_loss_total += loss
        print_val_loss_total += val_loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print_val_loss_avg = print_val_loss_total / print_every
            
            print('%s (%d %d%%) train_losses=%.4f val_losses=%.4f' % (timeSince(start, epoch / n_epochs),              #выводим среднюю ошибку на данном этапе
                                        epoch, epoch / n_epochs * 100,print_loss_avg,print_val_loss_avg))
            
            if (val_loss_best<=print_val_loss_total):
                if early_stop:
                    print("Training was interupted due to start of overfitting")
                    break
            else:
                val_lost_best=print_val_loss_total
                model_options_best=model.state_dict()
                
            print_val_loss_total = 0
    model.load_state_dict( model_options_best)
    print("Training finishid.")


<h3> Инициализация простой GRU

In [20]:
class GRU(nn.Module):
    def __init__(self, input_size=len(word2idx)+1, hidden_size=64, output_size=6, dropout_p=0.05):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)#На вход отправляется вектора размера input_size, которые мы кодируем в вектора размера hidden_size
        self.gru = nn.GRU(hidden_size, output_size, batch_first=True)# Инициализируем модель GRU (Gated Reccurent Unit, она имеет два типа воротЖ)
                                                                   # обновления и сброса, в отличие от 3 в LSTM), которая будет принимать вектора
                                                                    # размера hidden_size и имеет hidden_size скрытых слоё                                                                    #, параметр batch_first отвечает лишь за порядок вывода
      #  self.softmax=nn.Softmax(dim=1)
 #       self.dropout = nn.Dropout(dropout_p)  # модуль зануляющий каждый элемент тензора с вероятностью dropout_p, что благотовроно влияет на обучение
                                            # нейронной сети, путём предупреждения совместной адаптации нейрнов (Об этом стоит почитать побольше)
    def forward(self, input):
        embedded =(self.embedding(input))  # Прогон модели выглядит достаточно просто: имбедим(кодируем), зануляем часть,
   #     embedded= self.dropout(embedded)
        _, result = self.gru(embedded)            # пропускаем через GRU, которая на выходе даёт нам набор закодированных предложений и
       # result=self.softmax(result)
        return result.squeeze(0)      

<h3> Инициализация простой LSTM

In [108]:
class LSTM(nn.Module):
    def __init__(self, input_size=len(word2idx)+1, hidden_size=64, output_size=6, dropout_p=0.05):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)#На вход отправляется вектора размера input_size, которые мы кодируем в вектора размера hidden_size
        self.lstm = nn.LSTM(hidden_size, output_size, batch_first=True)# Инициализируем модель GRU (Gated Reccurent Unit, она имеет два типа воротЖ)
                                                                   # обновления и сброса, в отличие от 3 в LSTM), которая будет принимать вектора
                                                                    # размера hidden_size и имеет hidden_size скрытых слоё                                                                    #, параметр batch_first отвечает лишь за порядок вывода
      #  self.softmax=nn.Softmax(dim=1)
 #       self.dropout = nn.Dropout(dropout_p)  # модуль зануляющий каждый элемент тензора с вероятностью dropout_p, что благотовроно влияет на обучение
                                            # нейронной сети, путём предупреждения совместной адаптации нейрнов (Об этом стоит почитать побольше)
    def forward(self, input):
        embedded =(self.embedding(input))  # Прогон модели выглядит достаточно просто: имбедим(кодируем), зануляем часть,
   #     embedded= self.dropout(embedded)
        _, result = self.lstm(embedded)            # пропускаем через GRU, которая на выходе даёт нам набор закодированных предложений и
       # result=self.softmax(result)
        return result[0].squeeze(0)      

<h3> Инициализация простой RNN

In [111]:
class RNN(nn.Module):
    def __init__(self, input_size=len(word2idx)+1, hidden_size=64, output_size=6, dropout_p=0.05):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)#На вход отправляется вектора размера input_size, которые мы кодируем в вектора размера hidden_size
        self.rnn = nn.RNN(hidden_size, output_size, batch_first=True)# Инициализируем модель GRU (Gated Reccurent Unit, она имеет два типа воротЖ)
                                                                   # обновления и сброса, в отличие от 3 в LSTM), которая будет принимать вектора
                                                                    # размера hidden_size и имеет hidden_size скрытых слоё                                                                    #, параметр batch_first отвечает лишь за порядок вывода
      #  self.softmax=nn.Softmax(dim=1)
 #       self.dropout = nn.Dropout(dropout_p)  # модуль зануляющий каждый элемент тензора с вероятностью dropout_p, что благотовроно влияет на обучение
                                            # нейронной сети, путём предупреждения совместной адаптации нейрнов (Об этом стоит почитать побольше)
    def forward(self, input):
        embedded =(self.embedding(input))  # Прогон модели выглядит достаточно просто: имбедим(кодируем), зануляем часть,
   #     embedded= self.dropout(embedded)
        _, result = self.rnn(embedded)            # пропускаем через GRU, которая на выходе даёт нам набор закодированных предложений и
       # result=self.softmax(result)
        return result.squeeze(0)      

# Обучение

<h3> Получение весов функции потерь и даталоадеров

In [104]:
weights, train_dataloader,val_dataloader,test_dataloader = make_dataloader(batch_size=16) #получим данные

Число представителей каждого класса [35933, 14895, 2755, 2562, 1331, 2413]
Веса функции потерь tensor([0.0370, 0.0894, 0.4831, 0.5195, 1.0000, 0.5516])
tensor([[  28, 1527,    0,  ...,    0,    0,    0],
        [   8,    1,  784,  ...,    0,    0,    0],
        [   6,   21,    0,  ...,    0,    0,    0],
        ...,
        [  21,  708,  268,  ...,    0,    0,    0],
        [ 232,  671,   57,  ...,    0,    0,    0],
        [  70,  513,   21,  ...,    0,    0,    0]])


<h3> Обучение GRU

In [106]:
model = GRU().to(device) #инициализируем наши нейронки

train(train_dataloader,val_dataloader, model,loss_weights=weights, n_epochs=100, print_every=1)# начнём обучение

0m 34s (- 57m 1s) (1 1%) train_losses=1.7274 val_losses=1.6594
1m 10s (- 57m 53s) (2 2%) train_losses=1.5888 val_losses=1.5240
1m 48s (- 58m 22s) (3 3%) train_losses=1.4761 val_losses=1.4231
2m 22s (- 57m 10s) (4 4%) train_losses=1.3822 val_losses=1.3436
2m 57s (- 56m 5s) (5 5%) train_losses=1.3238 val_losses=1.2905
3m 31s (- 55m 15s) (6 6%) train_losses=1.2916 val_losses=1.2625
4m 6s (- 54m 32s) (7 7%) train_losses=1.2728 val_losses=1.3398
4m 40s (- 53m 49s) (8 8%) train_losses=1.3280 val_losses=1.3526
5m 15s (- 53m 9s) (9 9%) train_losses=1.3499 val_losses=1.3348
5m 50s (- 52m 34s) (10 10%) train_losses=1.3335 val_losses=1.3217
6m 25s (- 52m 2s) (11 11%) train_losses=1.3183 val_losses=1.3172
7m 0s (- 51m 23s) (12 12%) train_losses=1.3066 val_losses=1.2945
7m 35s (- 50m 46s) (13 13%) train_losses=1.2910 val_losses=1.2748
8m 9s (- 50m 8s) (14 14%) train_losses=1.2718 val_losses=1.2609
8m 44s (- 49m 32s) (15 15%) train_losses=1.2543 val_losses=1.2392
9m 19s (- 48m 55s) (16 16%) train_lo

<h4> Сохраним параметры модели

In [107]:
torch.save(model.state_dict(),"..\models\gru.pth")

<h3> Обучение LSTM

In [109]:
model1 = LSTM().to(device) #инициализируем наши нейронки

train(train_dataloader,val_dataloader, model1,loss_weights=weights, n_epochs=100, print_every=1,early_stop=False)# начнём обучение


0m 7s (- 12m 23s) (1 1%) train_losses=1.7993 val_losses=1.7907
0m 15s (- 12m 20s) (2 2%) train_losses=1.7898 val_losses=1.7884
0m 22s (- 12m 18s) (3 3%) train_losses=1.7881 val_losses=1.7869
0m 30s (- 12m 9s) (4 4%) train_losses=1.7869 val_losses=1.7854
0m 37s (- 12m 0s) (5 5%) train_losses=1.7854 val_losses=1.7838
0m 45s (- 11m 52s) (6 6%) train_losses=1.7812 val_losses=1.7748
0m 53s (- 11m 45s) (7 7%) train_losses=1.7557 val_losses=1.7432
1m 0s (- 11m 36s) (8 8%) train_losses=1.7350 val_losses=1.7260
1m 8s (- 11m 28s) (9 9%) train_losses=1.7204 val_losses=1.7301
1m 15s (- 11m 22s) (10 10%) train_losses=1.7065 val_losses=1.6988
1m 23s (- 11m 15s) (11 11%) train_losses=1.6952 val_losses=1.6912
1m 31s (- 11m 7s) (12 12%) train_losses=1.6860 val_losses=1.6786
1m 39s (- 11m 3s) (13 13%) train_losses=1.6819 val_losses=1.6718
1m 46s (- 10m 55s) (14 14%) train_losses=1.6586 val_losses=1.6332
1m 54s (- 10m 47s) (15 15%) train_losses=1.6207 val_losses=1.6019
2m 2s (- 10m 43s) (16 16%) train_lo

<h4> Сохраним параметры модели

In [110]:
torch.save(model1.state_dict(),"..\models\lstm.pth")

<h3> Обучение RNN

In [112]:
model2 = RNN().to(device) #инициализируем наши нейронки

train(train_dataloader,val_dataloader, model2,loss_weights=weights, n_epochs=100, print_every=1)# начнём обучение
torch.save(model2.state_dict(),"..\models\Rnn.pth")

0m 13s (- 22m 32s) (1 1%) train_losses=1.7889 val_losses=1.7787
0m 27s (- 22m 11s) (2 2%) train_losses=1.7798 val_losses=1.7742
0m 40s (- 21m 59s) (3 3%) train_losses=1.7734 val_losses=1.7689
0m 54s (- 21m 45s) (4 4%) train_losses=1.7698 val_losses=1.7653
1m 7s (- 21m 30s) (5 5%) train_losses=1.7648 val_losses=1.7624
1m 21s (- 21m 18s) (6 6%) train_losses=1.7644 val_losses=1.7602
1m 35s (- 21m 3s) (7 7%) train_losses=1.7619 val_losses=1.7580
1m 48s (- 20m 50s) (8 8%) train_losses=1.7583 val_losses=1.7570
2m 2s (- 20m 36s) (9 9%) train_losses=1.7577 val_losses=1.7511
2m 16s (- 20m 24s) (10 10%) train_losses=1.7552 val_losses=1.7491
2m 29s (- 20m 11s) (11 11%) train_losses=1.7530 val_losses=1.7529
2m 43s (- 19m 58s) (12 12%) train_losses=1.7527 val_losses=1.7519
2m 57s (- 19m 45s) (13 13%) train_losses=1.7544 val_losses=1.7507
3m 10s (- 19m 31s) (14 14%) train_losses=1.7590 val_losses=1.7512
3m 24s (- 19m 16s) (15 15%) train_losses=1.7526 val_losses=1.7499
3m 37s (- 19m 3s) (16 16%) trai

<h4> Сохраним параметры модели

In [113]:
torch.save(model2.state_dict(),"..\models\Rnn.pth")

# Оценка моделей

<h3>Подгрузим веса в наши модели

In [114]:
test_model_GRU=GRU()
test_model_GRU.load_state_dict(torch.load("..\models\gru.pth",weights_only=True))
test_model_LSTM=LSTM()
test_model_LSTM.load_state_dict(torch.load("..\models\lstm.pth",weights_only=True))
test_model_RNN=RNN()
test_model_RNN.load_state_dict(torch.load("..\models\Rnn.pth",weights_only=True))

<All keys matched successfully>

In [40]:
from sklearn.metrics import  classification_report

<h3> Создание функции оценки

In [155]:
def test_model(model,test_data,coef):
    true_answers=[]
    model_answers=[]
    for data,target in test_data:
        true_answers+=list(target)
       # print(true_answers,target)
        results=model(data)
        for result in results:
            result=list(result.detach())
            if max(result)<coef:
                model_answers+=[0]
            else:
                 model_answers+=[result.index(max(result))]
    print(len(model_answers))
    print(len(true_answers))
    print(classification_report(model_answers,true_answers))

<h3> Оценка GRU

In [156]:
test_model(test_model_GRU,test_dataloader,0.8)

47911
47911
              precision    recall  f1-score   support

           0       0.92      0.86      0.89     30476
           1       0.70      0.88      0.78      9436
           2       0.82      0.69      0.75      2596
           3       0.69      0.58      0.63      2451
           4       0.76      0.73      0.75      1101
           5       0.79      0.84      0.81      1851

    accuracy                           0.84     47911
   macro avg       0.78      0.76      0.77     47911
weighted avg       0.85      0.84      0.84     47911



<h3> Оценка LSTM

In [157]:
test_model(test_model_LSTM,test_dataloader,0)

47911
47911
              precision    recall  f1-score   support

           0       0.91      0.80      0.85     32318
           1       0.04      0.76      0.08       631
           2       0.01      0.76      0.03        38
           3       0.85      0.36      0.50      4869
           4       0.79      0.12      0.20      7177
           5       0.83      0.57      0.68      2878

    accuracy                           0.64     47911
   macro avg       0.57      0.56      0.39     47911
weighted avg       0.87      0.64      0.70     47911



<h3> Оценка RNN

In [159]:
test_model(test_model_RNN,test_dataloader,0)

47911
47911
              precision    recall  f1-score   support

           0       0.97      0.61      0.75     45231
           1       0.04      0.55      0.07       784
           2       0.00      0.12      0.01        76
           3       0.04      0.19      0.07       431
           4       0.19      0.19      0.19      1075
           5       0.03      0.18      0.05       314

    accuracy                           0.60     47911
   macro avg       0.21      0.31      0.19     47911
weighted avg       0.92      0.60      0.72     47911



<h3> В нашем случае, ьез всяких сомнений, GRU показала наилучший результат, который в целом можно считать приемлимым, учитывая засорённость входных данных. LSTM справилась похуже и не видит некотрые классы, RNN полностью провалила задачу

# HH.ru

## Получение вакансий

In [179]:
import requests
import json
import re

In [161]:
keyword = 'программист'
url = f'https://api.hh.ru/vacancies'

In [173]:
def get_vacancies(keyword, page=0):
    params = {
        'text': keyword,
        'page': page,
        'per_page': 100  # Максимум на страницу
    }
    response = requests.get(url, params=params)
    return response.json()


In [195]:
vacancies = []
page = 0

while len(vacancies) < 200:
    data = get_vacancies(keyword, page)
    if 'items' in data:
        vacancies.extend(data['items'])
        if len(data['items']) < 100:  # Если на странице меньше 100, значит закончились вакансии
            break
    page += 1

len(vacancies)

200

In [196]:
vacancies[0]

{'id': '112772957',
 'premium': False,
 'name': 'Frontend-разработчик',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '2759',
  'name': 'Ташкент',
  'url': 'https://api.hh.ru/areas/2759'},
 'salary': {'from': 6000000, 'to': None, 'currency': 'UZS', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'response_url': None,
 'sort_point_distance': None,
 'published_at': '2024-12-09T15:09:20+0300',
 'created_at': '2024-12-09T15:09:20+0300',
 'archived': False,
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=112772957',
 'insider_interview': None,
 'url': 'https://api.hh.ru/vacancies/112772957?host=hh.ru',
 'alternate_url': 'https://hh.ru/vacancy/112772957',
 'relations': [],
 'employer': {'id': '11635296',
  'name': 'ART VITALITY',
  'url': 'https://api.hh.ru/employers/11635296',
  'alternate_url': 'https://hh.ru/employer/11635296',
  'logo_urls': None,
  'vacancies_url': 'https://api.hh

## Обработка вакансий

In [211]:
vacancies2=[]

In [212]:
for i in range(len(vacancies)):
    info=vacancies[i]
    info=(info['name']+" "+info['employer']['name']+" "+str(info['snippet'])+" "+info['schedule']['name']+" "
            +str(info['professional_roles'])+" "+info['employment']['name'])
    vacancies2.append(info)

In [213]:
import pandas as pd

In [214]:
vacancies2[0]

"Frontend-разработчик ART VITALITY {'requirement': 'Опыт работы с Nuxt 3 (SSR, SSG, гибридный рендеринг). Отличное знание Vue 3 и его экосистемы. Уверенное владение CSS-фреймворками...', 'responsibility': 'Разработка и поддержка современного интерфейса для CRM/ERP системы. Внедрение адаптивного дизайна и кроссбраузерной совместимости. Интеграция с серверной частью системы...'} Удаленная работа [{'id': '96', 'name': 'Программист, разработчик'}] Полная занятость"

In [275]:
vac=pd.DataFrame(vacancies2,columns=['vac'])

In [276]:
vac.head(10)

,vac
0,Frontend-разработчик ART VITALITY {'requiremen...
1,Frontend-разработчик INNOVATION TEXNO SERVICE ...
2,Frontend-разработчик Junior Хагрид {'requireme...
3,Junior java разработчик dBridge {'requirement'...
4,Frontend-разработчик Aksoft ( Asanov Manas) {'...
5,"Junior front-end разработчик (TypeScript, Reac..."
6,Frontend-разработчик Ворлдвайд Мультисервисес ...
7,Strong Junior Backend разработчик NCRM GROUP {...
8,Разработчик Python в IT-компанию (Удаленно / Б...
9,Junior/Middle Frontend разработчик (React) Ham...


In [277]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ov00v\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ov00v\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [278]:
from nltk.tokenize import word_tokenize
import re

In [279]:
vac["vac"]=vac["vac"].apply(lambda x:
        word_tokenize(re.sub('[^а-яА-Яa-zA-Z ]+', '', x).lower().replace('highlighttext', '')))

In [280]:
vac.head(10)

,vac
0,"[frontendразработчик, art, vitality, requireme..."
1,"[frontendразработчик, innovation, texno, servi..."
2,"[frontendразработчик, junior, хагрид, requirem..."
3,"[junior, java, разработчик, dbridge, requireme..."
4,"[frontendразработчик, aksoft, asanov, manas, r..."
5,"[junior, frontend, разработчик, typescript, re..."
6,"[frontendразработчик, ворлдвайд, мультисервисе..."
7,"[strong, junior, backend, разработчик, ncrm, g..."
8,"[разработчик, python, в, itкомпанию, удаленно,..."
9,"[juniormiddle, frontend, разработчик, react, h..."


In [281]:
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()

In [282]:
vac["vac"]=vac["vac"].apply(lambda x:[ morph.normal_forms(w)[0] for w in x])

In [283]:
vac.head(10)

,vac
0,"[frontendразработчик, art, vitality, requireme..."
1,"[frontendразработчик, innovation, texno, servi..."
2,"[frontendразработчик, junior, хагрид, requirem..."
3,"[junior, java, разработчик, dbridge, requireme..."
4,"[frontendразработчик, aksoft, asanov, manas, r..."
5,"[junior, frontend, разработчик, typescript, re..."
6,"[frontendразработчик, ворлдвайда, мультисервис..."
7,"[strong, junior, backend, разработчик, ncrm, g..."
8,"[разработчик, python, в, itкомпания, удалённый..."
9,"[juniormiddle, frontend, разработчик, react, h..."


In [284]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ov00v\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [285]:
stopwords.append(["requirement","responsibility","name","id"])

In [286]:
vac["vac"]=vac["vac"].apply(lambda x: [w for w in x if w not in stopwords])

In [287]:
vac.head(10)

,vac
0,"[frontendразработчик, art, vitality, requireme..."
1,"[frontendразработчик, innovation, texno, servi..."
2,"[frontendразработчик, junior, хагрид, requirem..."
3,"[junior, java, разработчик, dbridge, requireme..."
4,"[frontendразработчик, aksoft, asanov, manas, r..."
5,"[junior, frontend, разработчик, typescript, re..."
6,"[frontendразработчик, ворлдвайда, мультисервис..."
7,"[strong, junior, backend, разработчик, ncrm, g..."
8,"[разработчик, python, itкомпания, удалённый, с..."
9,"[juniormiddle, frontend, разработчик, react, h..."


## Отдельно токенизация и сохранение

In [295]:
all_words = set([word for sublist in vac["vac"] for word in sublist])

In [296]:
all_words

{'разнообразный',
 'experience',
 'средний',
 'smmспециалист',
 'designers',
 'itwork',
 'раскрывать',
 'that',
 'mysqlpostgresmongo',
 'программирование',
 'подход',
 'просьба',
 'структура',
 'анкетирование',
 'дать',
 'билайн',
 'website',
 'миткома',
 'интерпретация',
 'писать',
 'резиновый',
 'релэкс',
 'старт',
 'fintech',
 'jdbc',
 'casemanagement',
 'целое',
 'серверный',
 'dimax',
 'год',
 'фронтэнд',
 'pytest',
 'hig',
 'прототипирование',
 'корзина',
 'участие',
 'use',
 'хранение',
 'производительность',
 'слабый',
 'soap',
 'менторинг',
 'андрей',
 'разнородный',
 'sputnik',
 'webразработка',
 'взаимодействие',
 'джампа',
 'обработка',
 'криптовалюта',
 'использовать',
 'активный',
 'предоставление',
 'изготовление',
 'нагрузка',
 'qsoft',
 'кроссфункциональный',
 'маркетинг',
 'both',
 'front',
 'graphql',
 'модель',
 'jmixcuba',
 'mobx',
 'дизайнмакет',
 'router',
 'инфраструктура',
 'графический',
 'стрессоустойчивость',
 'человек',
 'мультироторный',
 'позиция',
 'комм

In [301]:
all_words_dict={word:idx for idx,word in enumerate(all_words)}
len(all_words_dict)

1704

In [302]:
def convert_vacs(nested_words):
    return [all_words_dict[word] for word  in nested_words]
vac["vac"] = vac["vac"].apply(convert_vacs)

In [303]:
vac.head(10)

,vac
0,"[1117, 1243, 461, 1352, 1105, 283, 192, 1285, ..."
1,"[1117, 839, 1202, 1187, 1352, 1105, 283, 1694,..."
2,"[1117, 605, 798, 1352, 725, 1105, 283, 71, 151..."
3,"[605, 362, 1553, 484, 1352, 1105, 283, 29, 182..."
4,"[1117, 1466, 138, 118, 1352, 725, 1105, 29, 42..."
5,"[605, 710, 1553, 1694, 1192, 1259, 1352, 429, ..."
6,"[1117, 1666, 848, 1352, 181, 782, 263, 266, 29..."
7,"[782, 605, 510, 1553, 225, 1411, 1352, 1105, 2..."
8,"[1553, 1604, 243, 1153, 432, 477, 733, 292, 13..."
9,"[919, 710, 1553, 1192, 1305, 638, 1352, 1020, ..."


In [305]:
vac.to_pickle("../data/vacs.pkl")